# Analyze first dataset; emojis and sun/rain keywords for uk in 2020 

First overview analysis of tweets looking at:
- frequency of tweeting, time of day
- word clouds
- occurence of keywords in tweets

In addition, attempt at tokenizing tweets to obtain more informed word clouds and analysis essence of tweets.

Results include:
- generate cleaned file with tokenized tweets 'df_all_tokenized.csv'
    - removed urls, hashtags #TEXT, at sign @TEXT 
    - dealing with emojis is non-trivial, included string manipulation to replace emojis with their name or include emoji symbols
    - obvious bots contribute ~100_000 tweets (20%), remove them for now as expect strong bias


In [ ]:
# allows update of external libraries without need to reload package
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import glob
import zipfile
import pathlib
import logging
import json
import string

import numpy as np
import pandas as pd
import matplotlib.font_manager
import matplotlib.pyplot as plt
import wget

import wordcloud
import tweepy
import a2.twitter.downloader
import a2.dataset

In [ ]:
wget.download(
    "https://www.wfonts.com/download/data/2016/04/23/symbola/symbola.zip",
    out="/tmp/",
)
with zipfile.ZipFile("/tmp/symbola.zip", "r") as zip_ref:
    zip_ref.extractall("fonts")

In [ ]:
!ls ../data/tweets/gb_2020_rain_sun_vocab_emojis

In [ ]:
filepath = "../data/tweets/gb_2020_rain_sun_vocab_emojis/"
font_path = "../src/a2/data/font/Symbola.ttf"  # required to render emojis
figure_path = pathlib.Path("../figures/data/gb_2020_rain_sun_vocab_emojis/")
if not os.path.exists(figure_path):
    os.makedirs(figure_path)
path = os.path.abspath(filepath)  # use your path
all_files = glob.glob(os.path.join(path, "tweets_2020*.json"))

a2.dataset.load_dataset.load_tweets_dataset_from_jsons(all_files)

ds["created_at"] = (["index"], pd.to_datetime(ds.created_at).values)

In [ ]:
!head -2 /home/kristian/Projects/a2/data/tweets/gb_2020_rain_sun_vocab_emojis/tweets_2020_08.csv

In [ ]:
header = "🏔️ OR 🏔️ OR ☀️ OR ☀️ OR 🌞 OR ⛅ OR ⛈️ OR ⛈️ OR 🌤️ OR 🌤️ OR 🌥️ OR 🌥️ OR 🌦️ OR 🌦️ OR 🌧️ OR 🌧️ OR 🌨️ OR 🌨️ OR 🌩️ OR 🌩️ OR ☔ OR ⛄ OR blizzard OR cloudburst OR downpour OR drizzle OR flash flood OR flood OR flood stage OR forecast OR freezing rain OR hail OR ice storm OR lightning OR precipitation OR rain OR rain gauge OR rain shadow OR rainbands OR rain shower OR snow OR snow shower OR snowstorm OR sun OR sunny OR thunder OR thunderstorm"
keywords = header.split(" OR ")

## Sample tweets

In [ ]:
a2.dataset.utils_dataset.print_tweet_sample(ds)

In [ ]:
authors = a2.dataset.utils_dataset.print_tweet_authors(ds, n_sample=5)

In [ ]:
authors = a2.dataset.utils_dataset.print_tweet_authors(ds, n_sample=5, authors=authors)

In [ ]:
ds_no_bots = ds.where(
    (ds.source == "Twitter for iPhone")
    | (ds.source == "Twitter for Android")
    | (ds.source == "Instagram")
    | (ds.source == "Twitter for iPad")
    | (ds.source == "Twitter Web Client"),
    drop=True,
)

In [ ]:
authors_no_bots = a2.dataset.utils_dataset.print_tweet_authors(ds_no_bots, n_sample=5)

## Look at time: created_at

In [ ]:
ds.groupby("created_at.minute").count().plot.scatter(x="minute", y="author_id");

## Significant influx of new tweets at full hour (bots?!) -> look at source

In [ ]:
ds_full_hour = ds.where(ds.created_at.dt.minute == 0, drop=True)

In [ ]:
ds_full_hour.groupby("source").count().plot.scatter(y="source", x="tweet_id", size=20)
ax = plt.gca()
ax.tick_params(axis="x", labelrotation=0)
ax.set_xlabel("source")
ax.set_ylabel("count");

In [ ]:
ds.where(ds.source.str.contains(r"MeteoWare Plus+"), drop=True).groupby("created_at.minute").count().plot.scatter(
    x="minute", y="author_id"
);

In [ ]:
ds.where(ds.source.str.contains(r"pywws"), drop=True).groupby("created_at.minute").count().plot.scatter(
    x="minute", y="author_id"
);

In [ ]:
ds.where(ds.source.str.contains(r"Sandaysoft Cumulus"), drop=True).groupby("created_at.minute").count().plot.scatter(
    x="minute", y="author_id"
);

## Time distribution of tweets

In [ ]:
ds.groupby("created_at.hour").count().plot.scatter(x="hour", y="tweet_id");

In [ ]:
ds.groupby("created_at.dayofyear").count().plot.scatter(x="dayofyear", y="tweet_id");

In [ ]:
ds.groupby("created_at.month").count().plot.scatter(x="month", y="tweet_id");

In [ ]:
ds.groupby("author_id").count()["tweet_id"].plot.hist(bins=100)
ax = plt.gca()
ax.set_yscale("log")
ax.set_xlabel("occurence of author_id")
ax.set_ylabel("count");

In [ ]:
ds.groupby("source").count().plot.scatter(y="source", x="tweet_id", size=20)
ax = plt.gca()
ax.tick_params(axis="x", labelrotation=0)
ax.set_xlabel("source")
ax.set_ylabel("count");

## Most active individual users

In [ ]:
activity = ds.groupby("author_id").count().sortby("tweet_id", ascending=False)
user_activity_sorted_by_activity = activity["author_id"]
number_of_tweets = activity["tweet_id"]

In [ ]:
mask = number_of_tweets > 2000
most_active_users = user_activity_sorted_by_activity[mask].values
for user_id, n_tweets in zip(most_active_users, number_of_tweets[mask].values):
    user = a2.data_manipulation.twitter.downloader.get_user_from_userid(user_id)["data"]
    print(
        f'{user["name"]}, {user_id}, @{user["username"]}, {user["location"] if "location" in user else "?"} --> {n_tweets} tweets'
    )

In [ ]:
ds.where(ds.author_id.isin(3029396645), drop=True).groupby("created_at.minute").count().plot.scatter(
    x="minute", y="author_id", ax=plt.gca()
);

In [ ]:
for user_id in most_active_users:
    print(
        np.unique(ds.where(ds.author_id == user_id, drop=True).author_id),
        user_id,
    )

In [ ]:
plt.rcParams["figure.dpi"] = 100
plt.rcParams["font.size"] = 8
plt.rcParams["lines.markersize"] = 2
axes = []
n_variables = 4
n_x = len(most_active_users)
n_y = n_variables
fig, axs = plt.subplots(n_x, n_y, figsize=(n_y * 2, n_x * 2), constrained_layout=True)
for i, user_id in enumerate(most_active_users):
    ax = axs[i, 0]
    ds.where(ds.author_id == user_id, drop=True).groupby("created_at.minute").count().plot.scatter(
        x="minute", y="tweet_id", ax=ax, label=user_id
    )
    ax.legend()
    ax = axs[i, 1]
    ds.where(ds.author_id == user_id, drop=True).groupby("created_at.hour").count().plot.scatter(
        x="hour", y="tweet_id", ax=ax
    )
    ax = axs[i, 2]
    ds.where(ds.author_id == user_id, drop=True).groupby("created_at.dayofyear").count().plot.scatter(
        x="dayofyear", y="tweet_id", ax=ax
    )
    ax = axs[i, 3]
    ds.where(ds.author_id == user_id, drop=True).groupby("created_at.month").count().plot.scatter(
        x="month", y="tweet_id", ax=ax
    );

## Look at text

In [ ]:
ds.where((ds.author_id == 2411260615) & (ds.created_at.dt.month == 4), drop=True).groupby(
    "created_at.dayofyear"
).count().plot.scatter(x="dayofyear", y="tweet_id");

## Many tweets in April seem to be duplicates

In [ ]:
text = ds.where(
    (ds.author_id == 2411260615) & (ds.created_at.dt.dayofyear > 95) & (ds.created_at.dt.dayofyear < 120),
    drop=True,
).text.values
time = ds.where(
    (ds.author_id == 2411260615) & (ds.created_at.dt.dayofyear > 95) & (ds.created_at.dt.dayofyear < 120),
    drop=True,
).created_at.values
mask = np.argsort(time)

In [ ]:
_, index1 = np.unique(ds["text"], return_index=True)
_, index2 = np.unique(ds["created_at"], return_index=True)
mask = np.intersect1d(index1, index2)
print(
    f"total tweets: {ds.tweet_id.shape[0]}, unique tweets & times {mask.shape[0]}, unique number of texts {index1.shape[0]}, unique number of creation times {index2.shape[0]}"
)

In [ ]:
ds_unique = ds.sel(index=mask)

## Visualize unique dataset

In [ ]:
d = ds_unique
plt.rcParams["figure.dpi"] = 100
plt.rcParams["font.size"] = 8
plt.rcParams["lines.markersize"] = 2
axes = []
n_variables = 4
n_x = len(most_active_users)
n_y = n_variables
fig, axs = plt.subplots(n_x, n_y, figsize=(n_y * 2, n_x * 2), constrained_layout=True)
for i, user_id in enumerate(most_active_users):
    ax = axs[i, 0]
    d.where(d.author_id == user_id, drop=True).groupby("created_at.minute").count().plot.scatter(
        x="minute", y="tweet_id", ax=ax, label=user_id
    )
    ax.legend()
    ax = axs[i, 1]
    d.where(d.author_id == user_id, drop=True).groupby("created_at.hour").count().plot.scatter(
        x="hour", y="tweet_id", ax=ax
    )
    ax = axs[i, 2]
    d.where(d.author_id == user_id, drop=True).groupby("created_at.dayofyear").count().plot.scatter(
        x="dayofyear", y="tweet_id", ax=ax
    )
    ax = axs[i, 3]
    d.where(d.author_id == user_id, drop=True).groupby("created_at.month").count().plot.scatter(
        x="month", y="tweet_id", ax=ax
    );

## Remove three main sources for weather bots 

In [ ]:
ds_red = ds_unique.where(
    ~ds_unique.source.str.contains(r"pywws|MeteoWare Plus+|Sandaysoft Cumulus"),
    drop=True,
)

In [ ]:
d = ds_red
plt.rcParams["figure.dpi"] = 100
plt.rcParams["font.size"] = 8
plt.rcParams["lines.markersize"] = 2
axes = []
n_variables = 4
n_x = 2
n_y = n_variables
users = [d.author_id]
fig, axs = plt.subplots(n_x, n_y, figsize=(n_y * 2, n_x * 2), constrained_layout=True)
for i, user_id in enumerate(users):
    ax = axs[i, 0]
    d.where(d.author_id == d.author_id, drop=True).groupby("created_at.minute").count().plot.scatter(
        x="minute", y="tweet_id", ax=ax, label=None
    )
    ax.legend()
    ax = axs[i, 1]
    d.where(d.author_id == user_id, drop=True).groupby("created_at.hour").count().plot.scatter(
        x="hour", y="tweet_id", ax=ax
    )
    ax = axs[i, 2]
    d.where(d.author_id == user_id, drop=True).groupby("created_at.dayofyear").count().plot.scatter(
        x="dayofyear", y="tweet_id", ax=ax
    )
    ax = axs[i, 3]
    d.where(d.author_id == user_id, drop=True).groupby("created_at.month").count().plot.scatter(
        x="month", y="tweet_id", ax=ax
    );

## most popular days for tweeting

In [ ]:
most_popular_days = ds.groupby("created_at.dayofyear").count().sortby("tweet_id", ascending=False)
popular_days = most_popular_days["dayofyear"]
number_of_tweets_per_day = most_popular_days["tweet_id"]
mask = number_of_tweets_per_day > 3000
print(f"average number of tweets per day: {np.average(number_of_tweets_per_day.values)}")
for n, d in zip(number_of_tweets_per_day[mask].values, popular_days[mask].values):
    print(f"{n} tweets on day {d}")

## Wordcloud

### Full unique dataset 

In [ ]:
normal_word = r"(?:\w[\w']+)"
# 2+ consecutive punctuations, e.x. :)
ascii_art = r"(?:[{punctuation}][{punctuation}]+)".format(punctuation=string.punctuation)
# a single character that is not alpha_numeric or other ascii printable
emoji = r"(?:[^\s])(?<![\w{ascii_printable}])".format(ascii_printable=string.printable)
regexp = r"{normal_word}|{ascii_art}|{emoji}".format(normal_word=normal_word, ascii_art=ascii_art, emoji=emoji)
dir_name = os.path.dirname(__file__) if "__file__" in locals() else os.getcwd()
text = " ".join(ds_unique.text.values)
# Generate a word cloud image
# The Symbola font includes most emoji
wc = wordcloud.WordCloud(font_path=font_path, regexp=regexp, width=800, height=400).generate(text)

plt.imshow(wc)
plt.axis("off")
plt.show()

### Dataset without big bots

In [ ]:
normal_word = r"(?:\w[\w']+)"
# 2+ consecutive punctuations, e.x. :)
ascii_art = r"(?:[{punctuation}][{punctuation}]+)".format(punctuation=string.punctuation)
# a single character that is not alpha_numeric or other ascii printable
emoji = r"(?:[^\s])(?<![\w{ascii_printable}])".format(ascii_printable=string.printable)
regexp = r"{normal_word}|{ascii_art}|{emoji}".format(normal_word=normal_word, ascii_art=ascii_art, emoji=emoji)
dir_name = os.path.dirname(__file__) if "__file__" in locals() else os.getcwd()
text = " ".join(ds_red.text.values)
# Generate a word cloud image
# The Symbola font includes most emoji
wc = wordcloud.WordCloud(font_path=font_path, regexp=regexp, width=800, height=400).generate(text)

plt.imshow(wc)
plt.axis("off")
plt.show()

## Distribution of keywords in tweets
Unfortunately can only use emojis with cairo backend, which does not seem to plot inline...

In [ ]:
plt.close("all")
%matplotlib inline

text = " ".join(ds_red.text.values)
occurence = []
for k in keywords:
    occurence.append(text.count(k))

import matplotlib, mplcairo

matplotlib.use("module://mplcairo.tk")
prop = matplotlib.font_manager.FontProperties(fname=font_path)
plt.rcParams["axes.unicode_minus"] = False  # These two lines need to be set manually
plt.rcParams["font.family"] = prop.get_family()
fig, axs = plt.subplots(2, 1, figsize=(20, 10), constrained_layout=True)
for i, log in enumerate([True, False]):
    ax = axs[i]
    plot = ax.bar(np.arange(len(occurence)), occurence)
    labels = ["{}".format(x) for x in keywords]
    for rect1, label in zip(plot, labels):
        height = rect1.get_height()
        ax.annotate(
            label,
            (rect1.get_x() + rect1.get_width() / 2, height + 5),
            ha="center",
            va="bottom",
            fontsize=16,
            fontproperties=prop,
            rotation=90,
        )
    ax.tick_params(axis="x", labelrotation=90)
    ax.set_xlabel("keywords")
    ax.set_ylabel("counts")
    if log:
        ax.set_yscale("log")
plt.draw()
plt.show()
fig.savefig(figure_path / "word_count_all.pdf")

In [ ]:
%matplotlib inline
# switching back to default backend

## Tokenize words, also removing stopwords, urls,... with spacy package -> how to deal with emojis unclear (could replace, use pretrained model or include emojis as tokens + train)

In [ ]:
# poetry run python -m spacy download en_core_web_sm
import string
import re
import spacy
import contextualSpellCheck
import spacymoji
import xarray as xr


def remove_urls(text):
    text = re.sub(r"\S*https?:\S*", "", text, flags=re.MULTILINE)
    return text


def remove_hashtags(text):
    # remove strings starting with # or @
    return re.sub(r"@[A-Za-z0-9]+|#[A-Za-z0-9]+", "", text, flags=re.MULTILINE)


def remove_instagram_atsign(text):
    return re.sub(r"@\S*[A-Za-z0-9\s,]+$", "", remove_links(text), flags=re.MULTILINE)


def strip_links(text):
    link_regex = re.compile(
        "((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)",
        re.DOTALL,
    )
    links = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ", ")
    return text


def remove_links(text):
    link_regex = re.compile(
        "((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)",
        re.DOTALL,
    )
    links = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], "")
    return text


def replace_emojis_with_words(text, emojis_dic):
    # Function for converting emoticons into word
    for emot in emojis_dic.keys():
        text = re.sub(
            "{}".format(emot),
            "_".join(emojis_dic[emot].replace(":", "").split()),
            text,
            flags=re.MULTILINE,
        )
    return text


class Tokenizer:
    def __init__(self, add_emojis=False, add_spellchecking=False):
        self.nlp = spacy.load("en_core_web_sm")
        self.add_emojis = add_emojis
        self.add_spellchecking = add_spellchecking

        if self.add_emojis:
            emoji_config = {"merge_spans": False}
            self.nlp.add_pipe("emoji", first=True, config=emoji_config)

        df_emoji = pd.read_csv("../src/a2/data/emoji/emoji_df.csv")
        self.emojis = "".join(df_emoji.emoji.values)
        self.emojis_dic = {
            k: v for k, v in zip(df_emoji.emoji.values, df_emoji.name.values) if k not in ["*️⃣", "*⃣"]
        }  # latter cause error for re.sub

        if add_spellchecking:
            self.nlp.add_pipe("contextual spellchecker", config={"max_edit_dist": 5})

        self.punctuations = string.punctuation

        self.stopwords = spacy.lang.en.stop_words.STOP_WORDS

    def tokenize(self, sentence, source, return_string=False, replace_emojis=False):
        sentence = remove_hashtags(sentence)
        sentence = remove_urls(sentence)
        sentence = re.sub(r"([{}])([{}])".format(self.emojis, self.emojis), r"\1 \2", sentence)
        sentence = replace_emojis_with_words(sentence, self.emojis_dic)
        if source == "Instagram":
            sentence = remove_instagram_atsign(sentence)

        docs = self.nlp(sentence)
        if self.add_emojis:
            emojis = [token.text for token in docs if token._.is_emoji]
        else:
            emojis = []
        if self.add_spellchecking:
            tokens = docs._.outcome_spellCheck
            logging.debug(f"tokens outcome spellcheck: {tokens}")
            tokens = self.nlp(tokens)
        else:
            tokens = docs
        # Lemmatize each token and convert each token into lowercase
        tokens = [word.lemma_.lower().strip() if word.lemma_ != "PROPN" else word.lower_ for word in tokens]

        # Remove stopwords
        tokens = [word for word in tokens if word not in self.stopwords and word not in self.punctuations]

        # remove empty tokens
        tokens = [word for word in tokens if word != ""]

        if not return_string:
            return tokens + emojis
        else:
            sentence = " ".join(tokens)
            if self.add_emojis:
                sentence += " " + " ".join(emojis)
            return sentence

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # removes warning from hugging-face

In [ ]:
%%time
# parallelized but still quite inefficient... using 10_000 tweets here as already takes ~3 min
import multiprocessing as mp
from functools import partial

tokenizer = Tokenizer()
if __name__ == "__main__":
    _tokenize = partial(tokenizer.tokenize, return_string=True)
    with mp.Pool(processes=16) as pool:
        tokens = pool.starmap(
            _tokenize,
            zip(ds_red.text.values[:10000], ds_red.source.values[:10000]),
        )

## Visualize distribution of tokenized texts 

In [ ]:
ds_red = ds_red.isel(index=slice(0, 10000))  # only 10_000 tweets see above
ds_red["tokens"] = (["index"], tokens)

In [ ]:
text_tokenized = " ".join(ds_red.tokens.values)

In [ ]:
wc = wordcloud.WordCloud(font_path=font_path, regexp=regexp, width=800, height=400).generate(text_tokenized)
plt.imshow(wc)
plt.axis("off")
plt.show()

In [ ]:
dir_name

In [ ]:
plt.close("all")
%matplotlib inline

text = text_tokenized
keywords_extended = ["☀"] + keywords
occurence = []
for k in keywords_extended:
    occurence.append(text.count(k))

import matplotlib, mplcairo

matplotlib.use("module://mplcairo.tk")
prop = matplotlib.font_manager.FontProperties(fname=os.path.join(dir_name, "fonts", "Symbola.ttf"))
plt.rcParams["axes.unicode_minus"] = False  # These two lines need to be set manually
plt.rcParams["font.family"] = prop.get_family()
fig, axs = plt.subplots(2, 1, figsize=(20, 10), constrained_layout=True)
for i, log in enumerate([True, False]):
    ax = axs[i]
    plot = ax.bar(np.arange(len(occurence)), occurence)
    labels = ["{}".format(x) for x in keywords_extended]
    for rect1, label in zip(plot, labels):
        height = rect1.get_height()
        ax.annotate(
            label,
            (rect1.get_x() + rect1.get_width() / 2, height + 5),
            ha="center",
            va="bottom",
            fontsize=16,
            fontproperties=prop,
            rotation=90,
        )
    ax.tick_params(axis="x", labelrotation=90)
    ax.set_xlabel("keywords")
    ax.set_ylabel("counts")
    if log:
        ax.set_yscale("log")
fig.savefig(figure_path / "word_count_tokenized.pdf")

In [ ]:
%matplotlib inline
# switching back to default backend

In [ ]:
def save_file(ds, filename, delete_file=True):
    new_header = [
        header,
        "# dates: 2020-01-1T00:00:00.000Z-->2020-02-1T00:00:00.000Z",
    ]
    with open(filename, "x") as f:
        f.write(f"{new_header[0]}\n")
        f.write(f"{new_header[1]}\n")
        ds.to_csv(f)


save_file(ds_red.to_pandas(), os.path.join(filepath, "ds_all_tokenized.csv"))